In [1]:
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [96]:
#Test webu, vytvoreni nadpisu tabulky a zapsani prvnich metadat 
url = "https://wiki.uibk.ac.at/noscemus/A_Latin_Letter_containing_some_Animadversions_upon_Mr._Isaac_Newton,_his_Theory_of_Light"
response = requests.get(url)

if response.status_code == 200:
    soup = BeautifulSoup(response.content, "html.parser")
    table = soup.find("table", class_="wikitable")
    data = [item.get_text(strip=True) for item in table.find_all("td")]
    indices = [item.get_text(strip=True) for item in table.find_all("th")]
    metadata_tabulka = pd.DataFrame(columns=indices)
else:
    print("Chyba, response code je:", response.status_code)

In [97]:
metadata_tabulka

,Author,Full title,In,Year,Place,Publisher/Printer,Era,Form/Genre,Discipline/Content,Digital copies,Description,References,Cited in,How to cite this entry,Internal notes


In [101]:
%%time
# vytvoreni seznamu webovych stranek noscemu, na zaklade ktereho iteruj
url = ["https://wiki.uibk.ac.at/noscemus/_-_/index.php?title=Category:Works&pageuntil=De+curandis+vulneribus+sclopettorum#mw-pages", "https://wiki.uibk.ac.at/noscemus/_-_/index.php?title=Category:Works&pagefrom=De+curandis+vulneribus+sclopettorum#mw-pages", "https://wiki.uibk.ac.at/noscemus/_-_/index.php?title=Category:Works&pagefrom=Discursus+astronomicus+novissimus#mw-pages", "https://wiki.uibk.ac.at/noscemus/_-_/index.php?title=Category:Works&pagefrom=In+opus+revolutionum+Nicolai+Copernici+Torunnaei+dialogus#mw-pages", "https://wiki.uibk.ac.at/noscemus/_-_/index.php?title=Category:Works&pagefrom=Petri+Nonii+Salaciensis+opera#mw-pages",]
seznam =[]
for item in url:
    response = requests.get(item)
    soup = BeautifulSoup(response.content, "html.parser")
    tag = soup.find("div", class_="mw-category")
    seznam.extend(re.findall("(?<=href=\").*(?=\" title)", str(tag)))

CPU times: user 239 ms, sys: 33 ms, total: 272 ms
Wall time: 2.02 s


In [99]:
len(seznam)

994

In [95]:
#zapsani metadat ze vsech zaznamu v noscemus do pd.dataframu "metadata_tabulka"
for page in seznam:
    url = "https://wiki.uibk.ac.at"+page
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    table = soup.find("table", class_="wikitable")
    data = [item.get_text(strip=True) for item in table.find_all("td")]
    metadata_tabulka = pd.DataFrame([data])
metadata_tabulka 
## pouzit concat of vojty, prepsat komentare do anglictiny

KeyboardInterrupt: 